In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("LoadCSV").getOrCreate()
from pyspark.sql import Window
from pyspark.sql.functions import *

In [4]:
df = spark.read.csv('dataset.csv',
                   sep = ',',
                   header = True,
                   quote = '',
                   inferSchema = True #Automatically defines the type of the schema
                   )

In [5]:
df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- Occupation: integer (nullable = true)
 |-- purchase_1: integer (nullable = true)
 |-- purchase_2: integer (nullable = true)
 |-- purchase_3: integer (nullable = true)
 |-- purchase_4: integer (nullable = true)
 |-- purchase_5: integer (nullable = true)



In [6]:
df.show(5)

+---+----------+----------+----------+----------+----------+----------+
|age|Occupation|purchase_1|purchase_2|purchase_3|purchase_4|purchase_5|
+---+----------+----------+----------+----------+----------+----------+
| 52|        19|      3041|       532|       183|     21835|     12338|
| 53|         6|      3052|      3999|       872|      8704|     10117|
| 20|         8|      3466|      8520|       138|     18532|      5271|
| 50|         2|      1062|       524|      1677|     22875|      4637|
| 55|         8|      1416|      4021|      1358|     10176|      8943|
+---+----------+----------+----------+----------+----------+----------+
only showing top 5 rows



# Question 1

Find the average of the purchases (1 – till 5)

In [7]:
df.describe('purchase_1','purchase_2','purchase_3','purchase_4','purchase_5').show()

+-------+------------------+-----------------+------------------+-----------------+------------------+
|summary|        purchase_1|       purchase_2|        purchase_3|       purchase_4|        purchase_5|
+-------+------------------+-----------------+------------------+-----------------+------------------+
|  count|             21001|            21001|             21001|            21001|             21001|
|   mean| 2505.573639350507|5027.339555259274|1248.7371553735536|12457.99985714966|7496.7633446026375|
| stddev|1441.4610097028874|2872.855810742304| 724.9649386299643|7222.963503678821| 4319.169116931873|
|    min|                 0|                0|                 0|                2|                 0|
|    max|              5000|            10000|              2500|            24999|             15000|
+-------+------------------+-----------------+------------------+-----------------+------------------+



# Question 2

How many buyers between age 20-22 AND 30-34 AND 47-50 

In [8]:
df.filter(((df.age>=20) & (df.age<=22)) | ((df.age>=30) & (df.age<=34)) | ((df.age>=47) & (df.age<=50))).count()               # Age between 

4604

In [9]:
df.filter(((df.age>=20) & (df.age<=22)) | ((df.age>=30) & (df.age<=34)) | ((df.age>=47) & (df.age<=50))).show(5)   

+---+----------+----------+----------+----------+----------+----------+
|age|Occupation|purchase_1|purchase_2|purchase_3|purchase_4|purchase_5|
+---+----------+----------+----------+----------+----------+----------+
| 20|         8|      3466|      8520|       138|     18532|      5271|
| 50|         2|      1062|       524|      1677|     22875|      4637|
| 47|        15|      3625|      3929|        86|     14172|      2624|
| 32|        18|      2939|      1552|      1750|     11543|       825|
| 31|        15|       840|      9348|      1703|      4073|      1668|
+---+----------+----------+----------+----------+----------+----------+
only showing top 5 rows



# Question 3

What AGE has the max average purchases (1 – till 5) 

In [10]:
df_Question3_1 = df.withColumn( 'rec_avg' , 
                     (df['purchase_1'] +
                     df['purchase_2'] +
                     df['purchase_3'] +
                     df['purchase_4'] +
                     df['purchase_5']) / 5)

    
df_Question3_2 = df_Question3_1.groupby('age').agg({
                       'rec_avg': 'max'
                       })

df_Question3_3 = df_Question3_2.sort(col("max(rec_avg)").desc()).show(1)



+---+------------+
|age|max(rec_avg)|
+---+------------+
| 34|     10890.2|
+---+------------+
only showing top 1 row



# Question 4

What is the purchase averages (1 till 5) per Occupation?

In [11]:
df_Question4_1 = df.withColumn( 'rec_avg' , 
                     (df['purchase_1'] +
                     df['purchase_2'] +
                     df['purchase_3'] +
                     df['purchase_4'] +
                     df['purchase_5']) / 5)

    
df_Question4_2 = df_Question4_1.groupby('Occupation').agg({
                       'rec_avg': 'avg'
                       }).sort(col("Occupation").asc()).show()



+----------+------------------+
|Occupation|      avg(rec_avg)|
+----------+------------------+
|         1| 5748.066146341469|
|         2| 5779.394990723554|
|         3|5752.9392473118305|
|         4| 5753.337523105361|
|         5| 5726.559047619042|
|         6| 5750.322753346085|
|         7|5669.5613382899555|
|         8| 5817.084200385356|
|         9| 5763.983794089614|
|        10| 5814.509108527134|
|        11| 5776.030362389815|
|        12| 5611.849099526066|
|        13| 5700.724653739605|
|        14| 5883.002564102563|
|        15|  5786.97421203439|
|        16| 5744.028182701649|
|        17|5713.5999999999985|
|        18| 5742.624254473167|
|        19| 5654.114596554853|
|        20| 5773.070481927708|
+----------+------------------+



# Question 5

What are the min & max purchases, per Occupation per age?

In [12]:
df.select('age','Occupation',(min('purchase_1').over(Window.partitionBy('age','Occupation'))).alias("purchase_1_min"),
                             (min('purchase_2').over(Window.partitionBy('age','Occupation'))).alias("purchase_2_min"),
                             (min('purchase_3').over(Window.partitionBy('age','Occupation'))).alias("purchase_3_min"),
                             (min('purchase_4').over(Window.partitionBy('age','Occupation'))).alias("purchase_4_min"),
                             (min('purchase_5').over(Window.partitionBy('age','Occupation'))).alias("purchase_5_min")
         ).distinct().show(5)
                     
                     
df.select('age','Occupation',(max('purchase_1').over(Window.partitionBy('age','Occupation'))).alias("purchase_1_max"),
                             (max('purchase_2').over(Window.partitionBy('age','Occupation'))).alias("purchase_2_max"),
                             (max('purchase_3').over(Window.partitionBy('age','Occupation'))).alias("purchase_3_max"),
                             (max('purchase_4').over(Window.partitionBy('age','Occupation'))).alias("purchase_4_max"),
                             (max('purchase_5').over(Window.partitionBy('age','Occupation'))).alias("purchase_5_max")
         ).distinct().show(5)

+---+----------+--------------+--------------+--------------+--------------+--------------+
|age|Occupation|purchase_1_min|purchase_2_min|purchase_3_min|purchase_4_min|purchase_5_min|
+---+----------+--------------+--------------+--------------+--------------+--------------+
| 16|         1|           515|           172|           139|           357|           143|
| 16|         2|           516|           408|            88|            27|           953|
| 16|         3|            58|           302|            58|           119|           599|
| 16|         4|           148|           225|            11|          3449|           179|
| 16|         5|            32|          1878|           144|          1397|           592|
+---+----------+--------------+--------------+--------------+--------------+--------------+
only showing top 5 rows

+---+----------+--------------+--------------+--------------+--------------+--------------+
|age|Occupation|purchase_1_max|purchase_2_max|purchase_

In [13]:
df_Question5_1 = df.groupby('age','Occupation').agg({
                     'purchase_1': 'max',
                     'purchase_2': 'max', 
                     'purchase_3': 'max',
                     'purchase_4': 'max',
                     'purchase_5': 'max',
                       }).sort(col('age'),col('Occupation').asc())


df_Question5_2 = df.groupby('age','Occupation').agg({
                     'purchase_1': 'min',
                     'purchase_2': 'min', 
                     'purchase_3': 'min',
                     'purchase_4': 'min',
                     'purchase_5': 'min',
                       }).sort(col('age'),col('Occupation').asc())


df_Question5_1.select('age','Occupation',
                     when((col('max(purchase_1)') >= col('max(purchase_2)'))& (col('max(purchase_1)') >= col('max(purchase_3)'))& (col('max(purchase_1)') >= col('max(purchase_4)'))& (col('max(purchase_1)') >= col('max(purchase_5)')), col('max(purchase_1)'))
                    .when((col('max(purchase_2)') >= col('max(purchase_1)'))& (col('max(purchase_2)') >= col('max(purchase_3)'))& (col('max(purchase_2)') >= col('max(purchase_4)'))& (col('max(purchase_2)') >= col('max(purchase_5)')), col('max(purchase_2)'))
                    .when((col('max(purchase_3)') >= col('max(purchase_1)'))& (col('max(purchase_3)') >= col('max(purchase_2)'))& (col('max(purchase_3)') >= col('max(purchase_4)'))& (col('max(purchase_3)') >= col('max(purchase_5)')), col('max(purchase_3)'))
                    .when((col('max(purchase_4)') >= col('max(purchase_1)'))& (col('max(purchase_4)') >= col('max(purchase_2)'))& (col('max(purchase_4)') >= col('max(purchase_3)'))& (col('max(purchase_4)') >= col('max(purchase_5)')), col('max(purchase_4)'))                    
                    .when((col('max(purchase_5)') >= col('max(purchase_1)'))& (col('max(purchase_5)') >= col('max(purchase_2)'))& (col('max(purchase_5)') >= col('max(purchase_3)'))& (col('max(purchase_5)') >= col('max(purchase_4)')), col('max(purchase_5)')).alias(" Max Purchase ")).show()
                      
df_Question5_2.select('age','Occupation',                
                     when((col('min(purchase_1)') <= col('min(purchase_2)'))& (col('min(purchase_1)') <= col('min(purchase_3)'))& (col('min(purchase_1)') <= col('min(purchase_4)'))& (col('min(purchase_1)') <= col('min(purchase_5)')), col('min(purchase_1)'))
                    .when((col('min(purchase_2)') <= col('min(purchase_1)'))& (col('min(purchase_2)') <= col('min(purchase_3)'))& (col('min(purchase_2)') <= col('min(purchase_4)'))& (col('min(purchase_2)') <= col('min(purchase_5)')), col('min(purchase_2)'))
                    .when((col('min(purchase_3)') <= col('min(purchase_1)'))& (col('min(purchase_3)') <= col('min(purchase_2)'))& (col('min(purchase_3)') <= col('min(purchase_4)'))& (col('min(purchase_3)') <= col('min(purchase_5)')), col('min(purchase_3)'))
                    .when((col('min(purchase_4)') <= col('min(purchase_1)'))& (col('min(purchase_4)') <= col('min(purchase_2)'))& (col('min(purchase_4)') <= col('min(purchase_3)'))& (col('min(purchase_4)') <= col('min(purchase_5)')), col('min(purchase_4)'))                    
                    .when((col('min(purchase_5)') <= col('min(purchase_1)'))& (col('min(purchase_5)') <= col('min(purchase_2)'))& (col('min(purchase_5)') <= col('min(purchase_3)'))& (col('min(purchase_5)') <= col('min(purchase_4)')), col('min(purchase_5)')).alias(" Min Purchase ")).show()


+---+----------+--------------+
|age|Occupation| Max Purchase |
+---+----------+--------------+
| 16|         1|         23959|
| 16|         2|         23653|
| 16|         3|         24869|
| 16|         4|         24373|
| 16|         5|         20722|
| 16|         6|         24476|
| 16|         7|         20546|
| 16|         8|         23332|
| 16|         9|         24347|
| 16|        10|         23814|
| 16|        11|         23504|
| 16|        12|         24747|
| 16|        13|         23711|
| 16|        14|         21888|
| 16|        15|         24213|
| 16|        16|         24764|
| 16|        17|         22821|
| 16|        18|         24859|
| 16|        19|         24909|
| 16|        20|         24409|
+---+----------+--------------+
only showing top 20 rows

+---+----------+--------------+
|age|Occupation| Min Purchase |
+---+----------+--------------+
| 16|         1|           139|
| 16|         2|            27|
| 16|         3|            58|
| 16|         

# Question 6

How many buyers between ages 40-47 buy @purchase3 more than 2,200 and purchase4 less than 1,000 and purchase5 between 10,000-11,500?

In [14]:
df.filter((df.age>=40) &
          (df.age>=47) &
          (df.purchase_3>2200) &
          (df.purchase_4 <1000)  &
          (df.purchase_5 >=10000)  &
          (df.purchase_5 <=11500)).count()       

6

# Question 7

What is the max purchase (1 till 5) per age?

In [15]:
df_Question7_3 = df.groupby('age').agg({
                     'purchase_1': 'max',
                     'purchase_2': 'max', 
                     'purchase_3': 'max',
                     'purchase_4': 'max',
                     'purchase_5': 'max'
                       }).sort(col("age").asc()).show(55)

+---+---------------+---------------+---------------+---------------+---------------+
|age|max(purchase_3)|max(purchase_2)|max(purchase_5)|max(purchase_1)|max(purchase_4)|
+---+---------------+---------------+---------------+---------------+---------------+
| 16|           2495|           9985|          14773|           4983|          24909|
| 17|           2493|           9968|          14953|           4990|          24999|
| 18|           2500|           9997|          14969|           4991|          24975|
| 19|           2495|           9994|          15000|           4991|          24901|
| 20|           2497|           9984|          14985|           5000|          24924|
| 21|           2494|           9984|          14994|           4997|          24952|
| 22|           2497|           9999|          14983|           4984|          24994|
| 23|           2497|           9983|          14967|           4992|          24967|
| 24|           2493|           9973|          14995| 

# Question 8